In [7]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import multiprocessing as mp

import pickle
import joblib
import functools
import itertools
import time

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests

In [8]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

def try_request(req):
    while True:
        response = esi_client.request(req)
        
        if response.status == 200:
            return response.data
        
        print(response.status)
        print(response.header)
        time.sleep(5)

Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [3]:
import json

def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);
    
with open('universe/market_types.txt') as f:
    types = intd(f)
    
with open('universe/constellations.txt') as f:
    constellations = intd(f)

In [4]:
systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

root = 'Jita'
root_id = [v for v in systems if systems[v]["name"] == root][0]

# Limit to high-sec systems
subselect = [k for k in systems if systems[k]["security_status"] >= 0.5]
subgraph = systems_graph.subgraph(subselect)

# Limit to component connected to Jita
subselect = nx.node_connected_component(subgraph, root_id)
subgraph = systems_graph.subgraph(subselect)
print('High-sec has {} systems'.format(len(subgraph)))

system_ids = list(subgraph)
type_ids = list(types)

landmark_names = ['Jita', 'Amarr', 'Rens', 'Hek', 'Dodixie', 'Oursulaert', 'Tash-Murkon Prime', 'Agil'];
landmarks = [v["system_id"] for v in systems.values() if v["name"] in landmark_names]

@functools.lru_cache(maxsize=None)
def get_region_id(system_id):
    return constellations[systems[system_id]["constellation_id"]]["region_id"]

@functools.lru_cache(maxsize=None)
def get_station(station_id):
    esi_op = app.op['get_universe_stations_station_id'](station_id = station_id);
    return esi_client.request(esi_op).data

@functools.lru_cache(maxsize=None)
def in_highsec(station_id):
    try:
        return get_station(station_id).system_id in subgraph
    except:
        return False

High-sec has 972 systems


In [41]:
# Load orders from EVE
region_id = get_region_id(root_id)

print('Loading regions')
#region_ids = esi_client.request(app.op['get_universe_regions']()).data
region_ids = list(set(
    get_region_id(system_id) for system_id in subgraph
))

print('Loading market orders')

def requests_in_region(region_id):
    region = try_request(
        app.op['get_universe_regions_region_id'](region_id = region_id)
    )
    
    print('Loading orders in {} ({})'.format(region.name, region_id))
    
    response = None
    while True:
        response = esi_client.request(
            app.op['get_markets_region_id_orders'](region_id = region_id)
        )
        
        if response.status == 200:
            break
        
        time.sleep(0.1)
    
    n_pages = response.header.get('X-pages', [1])[0]

    p = mp.pool.ThreadPool(1)

    requests = [
        app.op['get_markets_region_id_orders'](region_id = region_id, page = page, order_type = 'all')
        for page in range(1, n_pages + 1)
    ]
    
    return requests

orders = {
    region_id : {
        type_id : list(orders)
        for (type_id, orders) in itertools.groupby(
            sorted(
                (
                    result
                    for request in tqdm(requests_in_region(region_id), desc = 'Pages')
                    for result in try_request(request)
                ),
                key = lambda x : x.type_id
            ),
            key = lambda x : x.type_id
        )
    }
    for region_id in tqdm(region_ids, desc = 'Regions')
}

Loading regions
Loading market orders


Loading orders in Sinq Laison (10000032)



Loading orders in The Citadel (10000033)



Loading orders in The Forge (10000002)



Loading orders in Genesis (10000067)


...


Loading orders in Khanid (10000049)



Loading orders in Tash-Murkon (10000020)



Loading orders in Kador (10000052)



Loading orders in Molden Heath (10000028)



Loading orders in Heimatar (10000030)


In [42]:
flat_orders = [
    {
        k : v
        for k,v in order.items()
    }
    for d in tqdm(orders.values())
    for os in d.values()
    for order in os
]

df = pd.DataFrame(flat_orders)
df = df.set_index('order_id')

In [44]:
df[df.type_id == 34]

,duration,is_buy_order,issued,location_id,min_volume,price,range,system_id,type_id,volume_remain,volume_total
order_id,,,,,,,,,,,
5622028402,90,False,2020-02-29T02:12:37+00:00,60011983,1,8.00,region,30002656,34,67466,67466
5462301996,90,False,2020-01-08T20:49:48+00:00,60001864,1,7.57,region,30002659,34,43969472,43969472
5465715946,90,False,2020-01-15T03:23:03+00:00,60001864,1,7.59,region,30002659,34,43756270,43756270
5458648271,90,False,2020-01-03T11:59:28+00:00,60001864,1,7.55,region,30002659,34,15561344,45106952
5599686666,90,False,2020-02-24T16:36:53+00:00,60011866,1,8.70,region,30002659,34,56120886,56120886
...,...,...,...,...,...,...,...,...,...,...,...
5498503626,90,True,2019-12-03T20:28:05+00:00,60015037,1,4.17,station,30012547,34,7535272,10000000
5348521393,90,True,2020-02-07T17:26:49+00:00,60004588,1,0.04,10,30002510,34,526647968,600000000
5620093472,90,True,2020-02-29T14:07:14+00:00,60004588,1,6.09,1,30002510,34,40226957,100000000


In [121]:
df[df.system_id == 30002697].reset_index().set_index(['system_id', 'type_id']).groupby(['system_id', 'type_id']).count().iloc[:,0]

system_id  type_id
30002697   2752       1
           2759       1
           2774       1
           2775       1
           2780       1
           2784       1
           2787       1
           2792       1
           2794       1
           2823       1
           15596      1
           15610      1
           15617      1
           15623      1
           15642      1
           16186      1
           17649      2
           17651      2
           17653      2
           17655      1
           17661      2
           17663      2
           21026      1
           27935      1
           29993      1
           30003      1
           30009      1
           30465      1
           30467      1
           30479      1
           34860      1
           36954      1
           36966      1
           37028      1
           37047      1
           37343      1
           37835      1
           41086      1
           41132      2
           44002      1
           46157     

In [45]:
import datetime

now = datetime.datetime.now()

df.to_csv('market/{}.csv'.format(now.strftime('%Y-%b-%d %H-%M')))
df.to_csv('market/current.csv')

In [72]:
df[df.duration == 90]

,duration,is_buy_order,issued,location_id,min_volume,order_id,price,range,system_id,type_id,volume_remain,volume_total
order_id,,,,,,,,,,,,
5604786202,90,False,2020-02-07T04:32:45+00:00,60010948,1,5604786202,250000.00,region,30002692,19980,1,1
5607213292,90,True,2020-02-27T23:09:20+00:00,60011866,1,5607213292,517.42,station,30002659,27401,4767,10000
5614331424,90,True,2020-02-22T06:26:02+00:00,1025824394754,1,5614331424,25300000.00,1,30002661,10213,3,3
5380335439,90,True,2020-02-28T09:16:01+00:00,60011866,1,5380335439,6590.59,region,30002659,21430,88474,99000
5590442672,90,True,2020-02-23T12:07:24+00:00,60011866,1,5590442672,1113.01,station,30002659,247,99894,100000
...,...,...,...,...,...,...,...,...,...,...,...,...
5597298425,90,True,2020-01-27T05:11:45+00:00,60004588,1,5597298425,1.00,region,30002510,48375,1000,1000
5374541783,90,True,2020-01-13T15:09:13+00:00,60004588,1,5374541783,31000128.98,solarsystem,30002510,47700,7,10
5597298536,90,True,2020-01-27T05:11:58+00:00,60004588,1,5597298536,1.00,region,30002510,48376,1000,1000


In [81]:
df.set_index(['type_id', 'order_id']).sort_index().index['type_id']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [66]:
regions = [
    (s, get_region_id(s))
    for s in tqdm(systems)
]

pd.DataFrame(regions, columns = ['system', 'region']).set_index('system').to_csv('universe/systems_region.csv')